In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [4]:
import sys
parent_path = os.path.abspath("..")
sys.path.append(parent_path)

In [5]:
import numpy as np 
import matplotlib.pyplot as plt
import torch
from torchseis import models as zoo
from test_hrnet import HRNet
from skimage.transform import resize

In [6]:
def cal_metrics(full, tiled, eps=1e-12):
    """
    Compute scale-invariant numerical equivalence metrics between
    full-volume and tiled execution outputs using NumPy.

    Parameters
    ----------
    full : np.ndarray
        Output from full-volume execution.
    tiled : np.ndarray
        Output from tiled execution.
    eps : float
        Small constant to avoid division-by-zero.

    Returns
    -------
    dict
        - rrmse: Relative RMSE
        - pe99_9: 99.9th percentile absolute error
        - pearson: Pearson correlation coefficient
    """

    # Flatten and cast to float64 for maximum precision
    full = full.astype(np.float64).ravel()
    tiled = tiled.astype(np.float64).ravel()
    std = np.abs(full - tiled)
    std = std.std()

    # Error
    diff = full - tiled
    abs_diff = np.abs(diff)

    # ----- 1. Relative RMSE -----
    rrmse = np.sqrt(np.mean(diff ** 2)) / (np.linalg.norm(full) + eps)

    # ----- 2. 99.9th percentile absolute error -----
    pe99_9 = np.percentile(abs_diff, 99.9)
    pe99_9 = pe99_9 / (np.abs(full).max() + 1e-12)

    # ----- 3. Pearson correlation -----
    f_c = full - np.mean(full)
    t_c = tiled - np.mean(tiled)
    std_f = np.std(f_c) + eps
    std_t = np.std(t_c) + eps
    pearson = np.mean(f_c * t_c) / (std_f * std_t)

    return {
        "rrmse": float(rrmse),
        "pe99.9": float(pe99_9),
        "pearson": float(pearson),
        "std": float(std)
    }


## data

In [7]:
f3d = np.load('/home/lijintao/data/f3sub.npy')[:, :512, 200:712]
f3d = resize(f3d, (512, 512, 512))

In [8]:
f3d = (f3d - f3d.mean()) / f3d.std()
f3d = f3d / np.abs(f3d).max()
print(f3d.min(), f3d.max(), f3d.mean(), f3d.std())


-1.0 0.99625194 4.874273e-07 0.27207863


In [9]:
f3d = f3d.astype(np.float16)

## FaultSeg3d

In [12]:
from torchseis import models as zoo
model = zoo.FaultSeg3d()
params = torch.load('../pretrained/fault/faultseg3d/faultseg3d-2020-70.pth', weights_only=True)
model.load_state_dict(params)
model = model.half().eval().cuda()
# f3dc = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

# with torch.no_grad():
#     ofull = model(f3dc, rank=3).cpu().numpy()

In [13]:
### full

# model = model.half().eval().cuda()
f3dc = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    ofull = model(f3dc[:, :, :448], ).cpu().numpy()

In [14]:
f3dc2 = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    otiled = model(f3dc2[:, :, :448], -1).cpu().numpy()

In [15]:
met = cal_metrics(ofull, otiled)

In [16]:
met

{'rrmse': 1.5635924462344567e-08,
 'pe99.9': 0.0002442598925254029,
 'pearson': 0.999999985343015,
 'std': 1.8206591735563508e-05}

## FaultSSL

In [18]:
model = zoo.FaultSSL(mode='precision')
params = torch.load('../pretrained/fault/faultssl/PrecisionFirst.pth', weights_only=True)
model.load_state_dict(params)
model = model.half().eval().cuda()


In [19]:
### full

f3dc = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    ofull = model(f3dc, 0).cpu().numpy()

In [20]:
f3dc2 = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    otiled = model(f3dc2, -1).cpu().numpy()

In [21]:
met = cal_metrics(ofull, otiled)
print(met)

{'rrmse': 2.8311011715221076e-07, 'pe99.9': 0.0005035400390619964, 'pearson': 0.9999945420038652, 'std': 0.00039148313938255365}


## ChannelSeg3d

In [23]:
model = zoo.ChannelSeg3d()
params = torch.load('../pretrained/channel/channelseg3d/channelseg3d.pth', weights_only=True)
model.load_state_dict(params)

<All keys matched successfully>

In [24]:
### full

model = model.half().eval().cuda()
f3dc = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    ofull = model(f3dc, 0).cpu().numpy()

In [25]:
f3dc2 = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    otiled = model(f3dc2, -1).cpu().numpy()

In [26]:
met = cal_metrics(ofull, otiled)
print(met)

{'rrmse': 0.0, 'pe99.9': 0.0, 'pearson': 0.9999999999983674, 'std': 0.0}


## FaultSeg3dPlus

In [12]:
model = zoo.FaultSeg3dPlus()
params = torch.load('../pretrained/fault/faultseg3dplus/faultseg3dplus.pth', weights_only=True)
model.load_state_dict(params)
model = model.half().eval().cuda()


In [13]:
f3dc = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    ofull = model(f3dc[:, :, :384, :384, :448], 0).cpu().numpy()

In [14]:
f3dc2 = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    otiled = model(f3dc2[:, :, :384, :384, :448], -1).cpu().numpy()

In [15]:
met = cal_metrics(ofull, otiled)
print(met)

{'rrmse': 3.1491944699182194e-09, 'pe99.9': 0.0, 'pearson': 0.9999995699372536, 'std': 1.2821828194129228e-05}


In [17]:
diff = np.abs(ofull - otiled).astype(np.float32)
print(diff.mean(), diff.std(), diff.max())

3.7518626e-07 1.2821791e-05 0.00048828125


## FaultSSL-IOU

In [18]:
model = zoo.FaultSSL()
params = torch.load('../pretrained/fault/faultssl/IOUFirst.pth', weights_only=True)
model.load_state_dict(params)
model = model.half().eval().cuda()

In [19]:
f3dc = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    ofull = model(f3dc[:, :, :384, :384, :384], 0).cpu().numpy()

In [20]:
f3dc2 = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    otiled = model(f3dc2[:, :, :384, :384, :384], -1).cpu().numpy()

In [21]:
met = cal_metrics(ofull, otiled)
print(met)

{'rrmse': 4.474938675494047e-06, 'pe99.9': 0.1333007812498667, 'pearson': 0.9994066079756118, 'std': 0.008135528693888363}


In [22]:
diff = np.abs(ofull - otiled)
print(diff.mean(), diff.std(), diff.max())

0.0004568 0.00814 0.759


## Wang25Channel

In [23]:
model = zoo.Wang25Channel()
params = torch.load('../pretrained/channel/wang25channel/wang25meandering.pth', weights_only=True)
model.load_state_dict(params)
model = model.half().eval().cuda()

In [24]:
f3dc = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    ofull = model(f3dc[:, :, :384, :384, :448], 0).cpu().numpy()

In [25]:
f3dc2 = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    otiled = model(f3dc2[:, :, :384, :384, :448], -1).cpu().numpy()

In [26]:
met = cal_metrics(ofull, otiled)
print(met)

{'rrmse': 4.845516484765486e-09, 'pe99.9': 0.0004882812499995117, 'pearson': 0.9999999964914316, 'std': 3.9143859628808994e-05}


In [27]:
diff = np.abs(ofull - otiled)
print(diff.mean(), diff.std(), diff.max())

2.44e-06 0.0 0.00586


## Karst

In [28]:
model = zoo.KarstSeg3d()
params = torch.load('../pretrained/karst/karstseg3d.pth', weights_only=True)
model.load_state_dict(params)
model = model.half().eval().cuda()

In [29]:
f3dc = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    ofull = model(f3dc[:, :, :384, :384, :448], 0).cpu().numpy()

In [30]:
f3dc2 = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    otiled = model(f3dc2[:, :, :384, :384, :448], -1).cpu().numpy()

In [31]:
met = cal_metrics(ofull, otiled)
print(met)

{'rrmse': 7.404071567089916e-09, 'pe99.9': 3.814697265621185e-05, 'pearson': 0.99999999812061, 'std': 5.917990305392169e-06}


In [32]:
diff = np.abs(ofull - otiled)
print(diff.mean(), diff.std(), diff.max())

2e-07 0.0 0.0004883


## Bi2021rgt

In [35]:
model = zoo.Bi21RGT3d()
params = torch.load('../pretrained/rgt/bi2021rgt/bi2021rgt.pth', weights_only=True)
model.load_state_dict(params)
model = model.half().eval().cuda()

In [36]:
f3dc = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    ofull = model(f3dc[:, :, :384, :384, :384], 0).cpu().numpy()


In [37]:
f3dc2 = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()

with torch.no_grad():
    otiled = model(f3dc2[:, :, :384, :384, :384], -1).cpu().numpy()

In [39]:
met = cal_metrics(ofull, otiled)
print(met)

{'rrmse': 7.384109044728471e-08, 'pe99.9': 0.0012914840325606379, 'pearson': 0.99999984562955, 'std': 0.0004895349064439228}


In [38]:
diff = np.abs(ofull - otiled)
print(diff.mean(), diff.std(), diff.max())

0.0002627 0.000423 0.006104


## DeepISMNet

In [10]:
model = zoo.DeepISMNet()
params = torch.load('../pretrained/ism/deepismnet.pth', weights_only=True)
model.load_state_dict(params)
model = model.half().eval().cuda()

In [11]:
f3dc = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()
f3dc = f3dc.repeat(1, 2, 1, 1, 1)

with torch.no_grad():
    ofull = model(f3dc[:, :, :256, :256, :320], 0).cpu().numpy()


In [12]:
f3dc2 = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()
f3dc2 = f3dc2.repeat(1, 2, 1, 1, 1)

with torch.no_grad():
    otiled = model(f3dc2[:, :, :256, :256, :320], 1).cpu().numpy()

In [13]:
f3dc3 = torch.from_numpy(f3d[np.newaxis, np.newaxis].copy()).half().cuda()
f3dc3 = f3dc3.repeat(1, 2, 1, 1, 1)

with torch.no_grad():
    otiled = model(f3dc3[:, :, :256, :256, :320], 2).cpu().numpy()

In [14]:
met = cal_metrics(ofull, otiled)
print(met)

{'rrmse': 3.265734132466593e-07, 'pe99.9': 0.0007462686567163704, 'pearson': 0.9999988784059102, 'std': 0.0017503297653744278}


In [15]:
diff = np.abs(ofull - otiled).astype(np.float32)
print(diff.mean(), diff.std(), diff.max())

0.001335128 0.0017503292 0.234375


## GEM

In [21]:
model = HRNet(base=64, use_checkpoint=False, drop_path_rate=0.2)
model = model.half().eval().cuda()

# params = torch.load('pretrained/channel/channelseg3d/channelseg3d.pth', weights_only=True)
# model.load_state_dict(params)

In [22]:
### full

f3dc = torch.from_numpy(f3d[np.newaxis, np.newaxis, :384, :384, :448].copy()).half().cuda()
f3dc = f3dc.repeat(1, 3, 1, 1, 1)

with torch.no_grad():
    ofull = model(f3dc).cpu().numpy()

In [23]:
f3dc2 = torch.from_numpy(f3d[np.newaxis, np.newaxis, :384, :384, :448].copy()).half().cuda()
f3dc2 = f3dc2.repeat(1, 3, 1, 1, 1)


with torch.no_grad():
    otiled = model.forward2(f3dc2, False).cpu().numpy()

In [24]:
met = cal_metrics(ofull, otiled)
print(met)

{'rrmse': 3.379729400663377e-06, 'pe99.9': 0.05957031249994042, 'pearson': 0.9996150570389523, 'std': 0.00575367593824745}


In [25]:
k = np.abs(ofull - otiled).astype(np.float32)
print(k.mean(), k.std())


0.0019773738 0.005753676
